<a href="https://colab.research.google.com/github/Bryncs/Challenge_Data_Science_2_Alura_Semana_1/blob/main/InsightPlaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto InsightPlaces



## Preparando Ambiente

### Preparando o Spark

In [38]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [39]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [40]:
import findspark
findspark.init()

In [41]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Projeto InsighPlaces") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [42]:
spark

### Preparando o Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Iniciando a Análise

### Importando os Dados

In [44]:
import zipfile

In [45]:
zipfile.ZipFile('/content/drive/MyDrive/insightPlaces/semana-1.zip', 'r').extractall('/content/drive/MyDrive/insightPlaces/')

In [46]:
path = '/content/drive/MyDrive/insightPlaces/dataset_bruto.json'
dados = spark.read.json(path)

## Entendendo o DataSet

[Base de dados - InsightPlaces](https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip)

#### Dicionário de Dados

| Colunas         | Descrição                                                      |
|-----------------|----------------------------------------------------------------|
| id              | Código de identificação do anúncio no sistema da InsightPlaces |
| tipo_unidade    | Tipo de imóvel (apartamento, casa e outros)                    |
| tipo_uso        | Tipo de uso do imóvel (residencial ou comercial)               |
| area_total      | Área total do imóvel (construção e terreno)                    |
| area_util       | Área construída do imóvel                                      |
| quartos         | Quantidade de quartos do imóvel                                |
| suites          | Quantidade de suítes do imóvel                                 |
| banheiros       | Quantidade de banheiros do imóvel                              |
| vaga            | Quantidade de vagas de garagem do imóvel                       |
| caracteristicas | Listagem de características do imóvel                          |
| andar           | Número do andar do imóvel                                      |
| endereco        | Informações sobre o endereço do imóvel                         |
| valores         | Informações sobre valores de venda e locação dos imóveis       |

### Estrutura da Base de Dados

In [47]:
dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [48]:
dados.show(5, truncate = False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------+
|anuncio                                                               

### Quantidade de Linhas e Colunas

In [49]:
linhas = dados.count()
colunas = len(dados.columns)

In [50]:
print(f'Este DataFrame contém {linhas} linhas e {colunas} colunas')

Este DataFrame contém 89083 linhas e 3 colunas


### Análise da Coluna Anúncios

In [51]:
# Seleção da coluna

dados_anuncio = dados.select('anuncio.*')
dados_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

In [52]:
dados_anuncio.show(10, truncate = False)

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |endereco                                                                                                                     |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+

### Filtrando a Base de dados

Tabelas de Frequência dos dados requisitados

In [54]:
colunas_frequencia = ['tipo_anuncio', 'tipo_unidade', 'tipo_uso']

In [55]:
for col in colunas_frequencia:
  dados_anuncio\
    .groupBy(col)\
    .count()\
    .orderBy('count', ascending = False)\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|88827|
|  Lançamento|  256|
+------------+-----+

+------------+-----+
|tipo_unidade|count|
+------------+-----+
| Apartamento|66801|
|      Outros|11963|
|        Casa|10319|
+------------+-----+

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|Residencial|84541|
|  Comercial| 4542|
+-----------+-----+



Filtros dos dados requisitados

In [56]:
dados_anuncio = dados_anuncio.filter("tipo_anuncio == 'Usado'")

In [57]:
dados_anuncio = dados_anuncio.filter("tipo_unidade == 'Apartamento'")

In [58]:
dados_anuncio = dados_anuncio.filter("tipo_uso == 'Residencial'")

In [60]:
dados_anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
|    2|      [42]|     [42]|      [1]|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{0, 0, Venda, 15...|
|    1|      [41]|     [41]|      [1]|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

### Listas para Inteiros

In [61]:
from pyspark.sql import functions as f

In [62]:
# Colunas com dados dentro de listas

listas = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

In [63]:
for col in listas:
  dados_anuncio\
      .select(f.size(f.col(col)).alias(col))\
      .groupBy(col)\
      .count()\
      .show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66562|
+-------+-----+

+------+-----+
|suites|count|
+------+-----+
|     1|61008|
|     0| 5554|
+------+-----+

+---------+-----+
|banheiros|count|
+---------+-----+
|        1|66562|
+---------+-----+

+----+-----+
|vaga|count|
+----+-----+
|   1|63545|
|   0| 3017|
+----+-----+

+----------+-----+
|area_total|count|
+----------+-----+
|         1|57368|
|         0| 9194|
+----------+-----+

+---------+-----+
|area_util|count|
+---------+-----+
|        1|66562|
+---------+-----+



In [64]:
# Retirando os dados de dentro das listas

dados_anuncio = dados_anuncio\
    .select([f.col(c)[0].alias(c) if c in listas else c for c in dados_anuncio.columns])

In [65]:
dados_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- eleme

In [66]:
dados_anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|
|    1|        41|       41|        1|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

### Retirando as colunas de dentro da coluna endereco



In [67]:
dados_anuncio = dados_anuncio.withColumn('bairro', dados_anuncio['endereco']['bairro'])
dados_anuncio.show(1)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|   bairro|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|Paciência|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+
only showing top 1 row



In [68]:
dados_anuncio = dados_anuncio.withColumn('zona', dados_anuncio['endereco']['zona'])
dados_anuncio.show(1)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|   bairro|      zona|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|Paciência|Zona Oeste|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+--

### Tratamento Coluna Valores

Inicialmente iremos retirar as colunas de dentro da coluna valores

In [69]:
dados_anuncio = dados_anuncio.withColumn('condominio', dados_anuncio['valores']['condominio'])
dados_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- eleme

In [70]:
dados_anuncio = dados_anuncio.withColumn('iptu', dados_anuncio['valores']['iptu'])
dados_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- eleme

In [71]:
dados_anuncio = dados_anuncio.withColumn('tipo', dados_anuncio['valores']['tipo'])
dados_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- eleme

In [72]:
dados_anuncio = dados_anuncio.withColumn('valor', dados_anuncio['valores']['valor'])
dados_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- eleme

In [74]:
# É possível observar que as colunas retiradas estão dentro de listas portanto iremos utilizar o método utilizado anteriormente

dados_anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+----------+------+-------+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|   bairro|      zona|condominio|  iptu|   tipo|  valor|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+----------+------+-------+-------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|Paciência|Zona Oeste|     [245]|[null]|[Venda]|[15000]|
|    2|        42|       42|        1|[Churrasqueira, P.

In [75]:
listas2 = ['condominio', 'iptu', 'tipo', 'valor']

In [76]:
for col in listas2:
  dados_anuncio\
      .select(f.size(f.col(col)).alias(col))\
      .groupBy(col)\
      .count()\
      .show()

+----------+-----+
|condominio|count|
+----------+-----+
|         1|65941|
|         2|  621|
+----------+-----+

+----+-----+
|iptu|count|
+----+-----+
|   1|65941|
|   2|  621|
+----+-----+

+----+-----+
|tipo|count|
+----+-----+
|   1|65941|
|   2|  621|
+----+-----+

+-----+-----+
|valor|count|
+-----+-----+
|    1|65941|
|    2|  621|
+-----+-----+



In [77]:
dados_anuncio = dados_anuncio\
    .select([f.col(c)[0].alias(c) if c in listas2 else c for c in dados_anuncio.columns])

In [78]:
dados_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- eleme

In [79]:
dados_anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+----------+----+-----+-----+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|   bairro|      zona|condominio|iptu| tipo|valor|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+----------+----+-----+-----+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|Paciência|Zona Oeste|       245|null|Venda|15000|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|

Selecionando apenas os valores de venda

In [80]:
dados_anuncio\
    .groupBy('tipo')\
    .count()\
    .show()

+-------+-----+
|   tipo|count|
+-------+-----+
|Aluguel|  214|
|  Venda|66348|
+-------+-----+



In [81]:
dados_anuncio = dados_anuncio.filter("tipo == 'Venda'")

In [82]:
dados_anuncio\
    .groupBy('tipo')\
    .count()\
    .show()

+-----+-----+
| tipo|count|
+-----+-----+
|Venda|66348|
+-----+-----+



In [83]:
dados_anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+----------+----+-----+-----+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|   bairro|      zona|condominio|iptu| tipo|valor|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+----------+----+-----+-----+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|Paciência|Zona Oeste|       245|null|Venda|15000|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|

## Salvando os dados

### Formato Parquet

In [85]:
%%time
dados_anuncio.write.parquet(
    path= '/content/drive/MyDrive/insightPlaces/parquet',
    mode= 'overwrite')

CPU times: user 45.9 ms, sys: 1.84 ms, total: 47.7 ms
Wall time: 7.46 s


### Formato CSV

Para salvar no formato CSV será necessário retirar as colunas endereco e valores e alterar o tipo da coluna caracteristicas, pois o formato não aceita dados do tipo array ou struct

In [86]:
from pyspark.sql.types import StringType

In [87]:
dados_anuncio = dados_anuncio.withColumn('caracteristicas', dados_anuncio['caracteristicas'].cast(StringType()))
dados_anuncio.select('caracteristicas').printSchema()

root
 |-- caracteristicas: string (nullable = true)



In [88]:
dados_anuncio = dados_anuncio.drop('endereco')
dados_anuncio = dados_anuncio.drop('valores')
dados_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



In [91]:
%%time
dados_anuncio.write.format("csv").save('/content/drive/MyDrive/insightPlaces/csv')

CPU times: user 23.6 ms, sys: 3.15 ms, total: 26.8 ms
Wall time: 3.49 s


### Comparativo Leitura dos DataSets salvos

Parquet

In [92]:
%%time
path = '/content/drive/MyDrive/insightPlaces/parquet'
dados = spark.read.parquet(path)

CPU times: user 5.52 ms, sys: 0 ns, total: 5.52 ms
Wall time: 434 ms


CSV

In [93]:
%%time
path = '/content/drive/MyDrive/insightPlaces/csv'
dados = spark.read.csv(path)

CPU times: user 8.58 ms, sys: 0 ns, total: 8.58 ms
Wall time: 462 ms
